In [2]:
from flask import Flask, request, jsonify, render_template
from ultralytics import YOLO
import cv2, os, re,subprocess
import numpy as np

In [6]:
os.chdir("../")

In [7]:
# Define the function to segment cracks
def segment_cracks(image_path, output_path):
    command = f"python predict.py --config configs/hrsegnetb32.yml --model_path hrsegnetb32/best_model/model.pdparams --image_path {image_path} --save_dir {output_path}"
    subprocess.run(command, shell=True)

In [8]:
filename = "data/inputImage.jpg"

In [9]:
model = YOLO('runs/segment/train2/weights/best.pt')

In [10]:
results = model.predict(source=filename,save=True)


image 1/1 d:\CrackedSegmentationModel\data\inputImage.jpg: 640x640 1 Cracks, 391.6ms
Speed: 14.8ms preprocess, 391.6ms inference, 1697.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict14


In [60]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'Cracks'}
 obb: None
 orig_img: array([[[176, 172, 161],
         [175, 171, 160],
         [173, 169, 158],
         ...,
         [168, 158, 148],
         [167, 157, 147],
         [162, 152, 142]],
 
        [[175, 171, 160],
         [174, 170, 159],
         [173, 169, 158],
         ...,
         [167, 157, 147],
         [165, 155, 145],
         [161, 151, 141]],
 
        [[176, 172, 161],
         [175, 171, 160],
         [174, 170, 159],
         ...,
         [166, 156, 146],
         [164, 154, 144],
         [160, 150, 140]],
 
        ...,
 
        [[178, 170, 157],
         [181, 173, 160],
         [183, 175, 162],
         ...,
         [158, 154, 143],
         [162, 158, 147],
         [163, 159, 148]],
 
        [[177, 169, 156],
         [180, 172, 159],
         [182, 174, 161

In [61]:
boxes = results.boxes
keypoints = results.keypoints
masks = results.masks
names = results.names
obb = results.obb
original_image = results.orig_img
original_shape = results.orig_shape
path = results.path
probabilities = results.probs
save_directory = results.save_dir
speed = results.speed


AttributeError: 'list' object has no attribute 'boxes'

In [64]:
result_object = results[0]
boxes = result_object.boxes
keypoints = result_object.keypoints
masks = result_object.masks
names = result_object.names
obb = result_object.obb
original_image = result_object.orig_img
original_shape = result_object.orig_shape
path = result_object.path
probabilities = result_object.probs
save_directory = result_object.save_dir
speed = result_object.speed

In [65]:
masks

ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
orig_shape: (500, 500)
shape: torch.Size([1, 640, 640])
xy: [array([[     3.9062,         150],
       [     3.9062,      150.78],
       [      3.125,      151.56],
       ...,
       [     7.8125,      151.56],
       [     7.0312,      151.56],
       [     5.4688,         150]], dtype=float32)]
xyn: [array([[  0.0078125,         0.3],
       [  0.0078125,     0.30156],
       [    0.00625,     0.30312],
       ...,
       [   0.015625,     0.30312],
       [   0.014062,     0.30312],
       [   0.010937,         0.3]], dtype=float32)]

In [69]:
import numpy as np

# Load the original image
original_image = results[0].orig_img

# Get the masks
masks = results[0].masks

# Initialize the total number of masked pixels
total_masked_pixels_count = 0

# If masks is a list of arrays
if isinstance(masks, list):
    for mask in masks:
        total_masked_pixels_count += np.count_nonzero(mask)

# If masks is a single array
elif isinstance(masks, np.ndarray):
    total_masked_pixels_count += np.count_nonzero(masks)

# Calculate the total number of pixels in the original image
total_pixels = np.prod(original_image.shape[:2])

# Calculate the percentage of the original image covered by masks
percentage_covered = (total_masked_pixels_count / total_pixels) * 100

print("Percentage of original image covered by masks: {:.2f}%".format(percentage_covered))


Percentage of original image covered by masks: 0.00%


In [31]:
# Attempt to read the image
input_image = cv2.imread(image_path)

# Check if the image was read successfully
if input_image is None:
    print("Error: Unable to read the image file.")
else:
    # Get the dimensions of the image
    height, width, _ = input_image.shape
    print("Image dimensions - Height:", height, "Width:", width)


Image dimensions - Height: 500 Width: 500


In [42]:
parent_directory = "runs/segment/"
directories = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]
pattern = "predict"
filtered_directories = [d for d in directories if d.startswith(pattern) and d[len(pattern):].isdigit()]
latest_directory = max(filtered_directories, key=lambda x: int(re.search(r'\d+$', x).group()))
image_path = os.path.join(parent_directory, latest_directory, "inputImage.jpg").replace('\\', '/')
image_path

'runs/segment/predict14/inputImage.jpg'

In [43]:
output_path = 'output/result'
if not os.path.exists(output_path):
     os.makedirs(output_path, exist_ok=True)

In [44]:
segment_cracks(image_path, output_path)


In [45]:
segmented_image = cv2.imread(image_path)

In [46]:
gray_image = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)

In [47]:
_, binary_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

In [48]:
kernel = np.ones((250, 250), np.uint8)

In [49]:
dilated_image = cv2.dilate(binary_image, kernel, iterations=1)

In [50]:
dilated_image

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [51]:
total_segmented_area = 0
segmented_area = np.count_nonzero(dilated_image) / (height * width) * 100
total_segmented_area += segmented_area
# Calculate the overall percentage of damage of the wall
overall_percentage_damage = total_segmented_area

print(f"Overall percentage of damage: {overall_percentage_damage:.2f}%")

Overall percentage of damage: 100.00%


In [ ]:
parent_folder = "hrseg/results"

# Create parent folder and subfolders if they don't exist
os.makedirs(parent_folder, exist_ok=True)
os.makedirs(os.path.join(parent_folder, "gray_scale"), exist_ok=True)
os.makedirs(os.path.join(parent_folder, "after_dilated"), exist_ok=True)

In [ ]:
output_dir = "hrseg/results/gray_scale"
output_dir1 = "hrseg/results/after_dilated"


cv2.imwrite(os.path.join(output_dir, "wall_image.jpg"), binary_image)
cv2.imwrite(os.path.join(output_dir1, "wall_image.jpg"), dilated_image)

True

In [ ]:
import base64
with open(os.path.join(output_dir1, "wall_image.jpg"), "rb") as img_file:
    encoded_img = base64.b64encode(img_file.read())

In [ ]:
result = {"image": encoded_img.decode('utf-8')}

In [ ]:
result

{'image': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/wAALCAH0AfQBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/AP38ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo